# Eager and Lazy Evaluation
- Eager - do tasks all at once
- Lazy - do tasks incrementally, on demand
- pros and cons to each method
- lazy advantages
    - don't need to store things until they are used
    - don't make more than you need
    - don't make things and then throw them away if not needed
    - can simulate infinite lists
- for loops can iterate over eager and lazy sequences

In [23]:
# 'for' will loop over an existing list

x = [3,5,'sadf', .343]
for e in x:
    print(e)

3
5
sadf
0.343


In [25]:
# range just evaluates to itself

r = range(4)
r

range(0, 4)

In [26]:
# a range holds the values of the original args

[r.start, r.stop]

[0, 4]

In [27]:
# can make a list out of it

list(range(0, 4))

[0, 1, 2, 3]

In [28]:
# for iterates over integers specified by range 

for x in range(4):
    print(x)


0
1
2
3


In [29]:
# note: here range specifies the iteration, 
# but a million element list is never created

sum = 0
for x in range(1000000):
    sum += x
sum

499999500000

In [30]:
# here the list that range specifies IS created

rl = list(range(4))

print(rl)

for x in rl:
    print(x)

[0, 1, 2, 3]
0
1
2
3


# Iterator Protocol
- there is a general protocol for iterating over objects
- use 'iter' function to get an iterator from an object
    - not all objects have iterators - for example, int and float don't
- the 'iterator' may be the same object, or a different one
- some objects allow multiple iterators simultaneously
- call 'next' function repeatedly, to get the elements of the iteration
- when all elements have been produced, iterator will raise a 'StopIteration' error each
time 'next' is called
- 'StopIteration' implies the iterator is 'exhausted' - discard it.
- why raise an error at the end of the iteration???
- for loops use iterator protocol

In [31]:
x = [1,4]
xi = iter(x)
xi

In [32]:
# 1st value

next(xi)

1

In [33]:
# 2nd value

next(xi)

4

In [34]:
# done

next(xi)

StopIteration: 

In [35]:
# 'range' each iterator is a new obj - can have any number of them

r = range(2)
ri = iter(r)
ri2 = iter(r)

[r, ri, ri2, ri is r, ri is ri2]

[range(0, 2),
 False,
 False]

In [36]:
next(ri)

0

In [37]:
[next(ri), next(ri2)]

[1, 0]

In [38]:
# now ri is ahead of ri2, so it finishes first
next(ri)

StopIteration: 

In [39]:
# one val left for ri2

next(ri2)

1

In [40]:
# now ri2 is done

next(ri2)

StopIteration: 

# Generator Function
- one way to define an iterator
- a generator is defined by using a 'yield' statement inside a 'def'
- executing the function returns the iterator
- falling off the end of the function, or executing a 'return' statement, will terminate the generator.
- once a generator terminates, it is 'exhausted', and can not be used again
- calling 'next' on a generator will cause the generator to execute until it 
hits a 'yield' statement. The arg supplied to 'yield' will be returned by 'next'. The next time 'next' is called on the generator, the generator will resume executing on the statement following the yield. 
- all local variable values are preserved between between 'next' calls to the generator

In [41]:
# executing this function will return a generator

def gf(n):
    for j in range(n):
        yield(j)

In [42]:
# 'list' will run generator until it is exhausted.

list(gf(5))

[0, 1, 2, 3, 4]

In [43]:
# or can use returned generator explicitly via iteration protocol

g = gf(2)
g

<generator object gf at 0x10736c258>

In [44]:
next(g)

0

In [45]:
next(g)

1

In [46]:
# generator is finished - discard it

next(g)

StopIteration: 

In [47]:
# iterate over generator directly

[j+10 for j in gf(3)]

[10, 11, 12]

# A generator can represent an infinite sequence (sort of)
- can't make a list of all the even integers
- but in some sense can represent that list with a generator

In [48]:
def evens():
    e = 0
    # this generator will never terminate
    while True:
        e += 2
        yield(e)

In [49]:
eg = evens()
[next(eg), next(eg), next(eg)]

[2, 4, 6]

In [50]:
# a generator can use another generator

def evenPowersOf2():
    eg = evens()
    while True:
        e = next(eg)
        yield 2**e

In [51]:
ep = evenPowersOf2()

for p in ep:
    if p > 1040:
        break
    print(p)



4
16
64
256
1024


# Yields do not have to be inside a loop
- fibonacci series is 1,1,2,3,5,8...
- f(0) = 1
- f(1) = 1
- f(n) = f(n-1) + f(n-2)

In [52]:
def fibonacci():
    # easy way to handle the first two ones
    yield(1)
    yield(1)
    last = 1
    last2 = 1
    while True:
        sum = last + last2
        yield(sum)
        last2 = last
        last = sum

f = fibonacci()

for j in range(10):
    print( next(f))


1
1
2
3
5
8
13
21
34
55


# Modifying a Running Generator
- can change generator state at any time

In [53]:
def counter(maximum):
    cnt = 0
    while cnt < maximum:
        # peculiar syntax
        val = (yield cnt)
        # If value provided, change counter
        if val is not None:
            cnt = val
        else:
            cnt += 1


In [54]:
c = counter(1000)
[next(c) for j in range(10)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [55]:
# change the 'cnt' variable that the generator saves 
# '(yield cnt)' in generator will return 300

c.send(300)

# generator continues from new value
[next(c) for j in range(10)]

[301, 302, 303, 304, 305, 306, 307, 308, 309, 310]

In [56]:
# the generator is nowhere near done, but we can terminate it

c.close()

In [57]:
# the generator is exhausted now

next(c)

StopIteration: 

# Generator Expression
- an expression that evaluates to a generator
- looks like a list comprehension, but with outer '()' instead of '[]'

In [58]:
def ge(n):
    # can't return a 'def'
    return ( j**2 for j in range(2, n) if j != 3)

g = ge(8)

In [59]:
# pick first two manually

next(g)

4

In [60]:
# skipped j == 3

next(g)

16

In [61]:
# for gets the rest

for j in g:
    print(j)

25
36
49


# A generator will finish if it calls a generator that finishes

In [62]:
def g(n):
    for j in range(n):
        yield j

def g2(n):
    gen = g(n)
    while True:
        yield next(gen)


In [63]:
list(g2(3))

[0, 1, 2]